<a href="https://colab.research.google.com/github/JonathanSum/Deep-Learning-Happy-Sugar-Life-Weekly-Training/blob/master/play_char.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Train a character-level GPT on some text data

The inputs here are simple text files, which we chop up to individual characters and then train GPT on. So you could say this is a char-transformer instead of a char-rnn. Doesn't quite roll off the tongue as well. In this example we will feed it some Shakespeare, which we'll get it to predict character-level.

In [1]:
!git clone https://github.com/JonathanSum/minGPT.git

Cloning into 'minGPT'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 186 (delta 0), reused 0 (delta 0), pack-reused 182
Receiving objects: 100% (186/186), 1.38 MiB | 2.97 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [2]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [5]:
# make deterministic
from minGPT.mingpt.utils import set_seed
# set_seed(42)

In [6]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [7]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y


In [8]:
block_size = 128 # spatial extent of the model for its context

In [9]:
#minGPT.
!git clone https://github.com/JonathanSum/MyDataSet.git

Cloning into 'MyDataSet'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 10 (delta 1), reused 7 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [10]:
# you can download this file at https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt
text = open('MyDataSet/happy_sugar_life_text.txt', 'r').read() # don't worry we won't run out of file handles
train_dataset = CharDataset(text, block_size) # one line of poem is roughly 50 characters

data has 72549 characters, 72 unique.


In [11]:
from minGPT.mingpt.model import GPT, GPTConfig
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=512)
model = GPT(mconf)

09/28/2020 14:44:39 - INFO - minGPT.mingpt.model -   number of parameters: 2.535936e+07


In [12]:
from minGPT.mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=2, batch_size=8, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                      num_workers=4)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()

100%|██████████| 9053/9053 [07:06<00:00, 21.23it/s]


In [ ]:
# import gc
# trainer = None
# gc.collect()

1005

In [13]:
# alright, let's sample some character-level Shakespeare
from minGPT.mingpt.utils import sample

context = "Love"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 2000, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

Love...
Love... Love...
Love is different for every person.
The way it shines is different.
But anyone can receive it.
Once you experience it, your heart feels fulfilled in an instant.
I'm coming!
Welcome! I was waiting for you.
It'll be fine!
Love is something that can't be seen.
Love...
Auntie, you're hurt again.
-Are you okay? -Yeah.
I'm okay.
This is also love.
There are many different flavors of love.
Strawberry, lemon, apple, and mint.
There are also poisonous ones.
Taste it. Swallow it.
As long as there's some in the jar.
When they run out, you add more.
Over and over again.
When I do that, I feel surrounded by love,
and I feel happy.
But, my heart still had a big hole in the middle
all along.
Auntie. Will you hear me out?
What is it?
I killed someone.
For someone I love.
I’ll destroy the evidence to pursue love,
and travel far away, just the two of us. Help us out.
Satou-chan, you're so amazing.
Are you calling the police?
No.
Some of the people that came here
also did things l

In [ ]:
# well that was fun